In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
ejscreen_2021 = '/Users/butler/Documents/hp/Capstone_Two/data/raw/EJSCREEN_2020_StatePctile.csv'
df = pd.read_csv('/Users/butler/Documents/hp/Capstone_Two/notebooks/EJSCREEN_2021_StatePctile.csv')



In [ ]:
df.shape


In [ ]:
df.head()


In [ ]:
df.tail()


In [ ]:
df.describe()


In [ ]:
df.info()


In [ ]:
df.dtypes


In [ ]:
df['ACSTOTPOP'].dtypes


In [ ]:
df['MINORPCT'].dtypes


In [ ]:
df['PRE1960PCT'].dtypes


In [ ]:
df['P_MINORPCT'].dtypes


In [ ]:
pol = df.astype({"ID":'int64'})


In [ ]:
pol.head()


In [ ]:
df.columns.get_loc('P_MINORPCT')


 To prepare this for concatenation with health data, need to make health LocationName and df ID the same. ID has one extra digit for the block group, while LocationName only includes census tracts. Combining the block groups into census tracts by averaging the features scaled by population.

In [ ]:
pol['LocationName'] = pol['ID'] // 10
poll = pol.copy()
poll.head()


 Getting rid of any rows with zero population

In [ ]:
poll = poll[poll['ACSTOTPOP'] != 0] 


 Dropping columns not needed

In [ ]:
poll = poll.drop(["OBJECTID", 'STATE_NAME', 'ID', 'REGION', 'Shape_Length', 'Shape_Area'], axis=1)
poll.head()


In [ ]:
poll.head()


In [ ]:
df.columns.get_loc('DSLPM')


 Getting rid of map data

In [ ]:
df = poll
pop = df.ACSTOTPOP
loc = df.LocationName
df = df.iloc[:, 1:67]  


 Multiplying numerical columns, except ACSTOTPOP, by population (ACSTOTPOP), as a step in the process of combining the numberical data of rows with same LocationName by a weighted average

In [ ]:
df.head()


In [ ]:
df = df.drop(['B_RESP', 'P_RESP', 'P_CANCR', 'B_CANCR', 'P_DSLPM', 'B_DSLPM', 'P_OZONE', 'B_OZONE', 'P_PM25', 'B_PM25','P_PWDIS_D2','B_PWDIS_D2', 'D_PWDIS_2','P_DSLPM_D2', 'B_DSLPM_D2','B_PWDIS','P_PTSDF', 'B_PTSDF','P_PRMP','B_PRMP','P_PNPL','B_PNPL','P_LDPNT','B_LDPNT','P_PTRAF','B_PTRAF','B_CANCR_D2'], axis=1)
df.head()


In [ ]:
df.columns


In [ ]:
pop = df['ACSTOTPOP']


In [ ]:
df = df.iloc[:,28:]


In [ ]:
df.head()


In [ ]:
df['ACSTOTPOP'] = pop
df.head()


In [ ]:
for column in df:
    if column != 'ACSTOTPOP':
        df[column] *= df['ACSTOTPOP']


 Adding back in LocationName

In [ ]:
df['LocationName'] = loc


 Grouping rows with same LocationName. Each numerical value is given the population-weighted mean

In [ ]:
def mean_normed_by_pop(x):
    x = x.sum() / x['ACSTOTPOP'].sum()
    return x

df = df.groupby('LocationName', group_keys=False).apply(mean_normed_by_pop)


 LocationName and ACSTOPOP were also normed, so they need to be dropped. The LocationName as the index is made into a column

In [ ]:
df.drop(['LocationName', 'ACSTOTPOP'], axis=1, inplace=True)
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'LocationName'})
df.head()


In [ ]:
df.tail()


In [ ]:
df.to_csv('/Users/butle/hp/Capstone_Two/data/interim/EJSCREEN_2020_interim.csv')


In [ ]:

health_2022 = '/Users/butler/Documents/hp/Capstone_Two/data/raw/PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2022_release.csv'
df2 = pd.read_csv(health_2022)


In [ ]:
df2.head()


In [ ]:
df2.shape


In [ ]:
df2.tail()


In [ ]:
df2 = df2[df2['Category'] == 'Health Outcomes']


In [ ]:
df2.shape


In [ ]:
df[df['LocationName'] == 1001020200]


In [ ]:
df[df['LocationName'] == 55133200400]


In [ ]:
df2['LocationName'].isin(df['LocationName']).value_counts()
    


In [ ]:
df['LocationName'].isin(df2['LocationName']).value_counts()


 Not all of the LocationNames in the health data are present in the environmental data. Slicing the health database

In [ ]:
common_rows_health = df2.loc[df2['LocationName'].isin(df['LocationName'])]
common_rows_health.head()


In [ ]:
common_rows_health.shape


In [ ]:
df3 = common_rows_health
df3.Category.nunique()


In [ ]:
df3.Category.unique()


In [ ]:
df3.shape


In [ ]:
df3.DataSource.nunique()


In [ ]:
df3.DataSource.unique()


In [ ]:
df3.Data_Value_Footnote_Symbol.nunique()


In [ ]:
df3.Data_Value_Footnote.nunique()


In [ ]:
df3.DataValueTypeID.unique() 


In [ ]:
df3.DataValueTypeID.nunique() 


 Getting rid of unnecessary columns

In [ ]:
df3.head()


In [ ]:
df3.shape


In [ ]:
df3.MeasureId.unique()


In [ ]:
df3.drop(['StateDesc', 'Year', 'CountyFIPS', 'Geolocation', 'DataValueTypeID', 'Data_Value_Footnote','Data_Value_Footnote_Symbol', 'DataSource'], axis=1, inplace=True)


In [ ]:
df3.head()


In [ ]:
df3 = df3.iloc[:, 2:] 


In [ ]:
df3.head()


In [ ]:
df3['Crude_prevalence'] = df3['Data_Value']


In [ ]:
df3['Category'].unique()


In [ ]:
df3.shape


In [ ]:
df3.head()


In [ ]:
df3 = df3.drop(['Category', 'Measure','Data_Value_Unit','Data_Value','LocationID','CategoryID'],axis=1)


In [ ]:
df3.head()


In [ ]:
df3.tail()


In [ ]:
df3 = df3.drop(['Data_Value_Type', 'Short_Question_Text'], axis=1)


In [ ]:
df3.columns


In [ ]:
col = df3.columns.to_list()
col


In [ ]:
cols = ['LocationName', 'MeasureId', 'Crude_prevalence','Low_Confidence_Limit', 'High_Confidence_Limit',  'TotalPopulation'  ]
cols


In [ ]:
df3 = df3[cols]
df3.columns


In [ ]:
df3.head()


In [ ]:
df3[df3['MeasureId'] == 'COPD'].shape


In [ ]:
df3[df3['MeasureId'] == 'Obesity'].shape


In [ ]:
df3.MeasureId.unique() 


In [ ]:
outcomes = df3.copy()


In [ ]:
combined = pd.merge(outcomes,df)
combined.rename({'LocationName': 'CensusTract', 'TotalPopulation':'TotalPopHealth',"CANCER": "AirToxicsRisk", 'DSLPM':'DieselPM'}, axis=1, inplace=True)
combined.head()


 Re-indexing as CensusTract will not be a feature.

In [ ]:
combined.set_index('CensusTract', inplace=True)


In [ ]:
combined.to_csv()


 Making a separate dataframe for each health concern.

In [ ]:
asthma = combined[combined['MeasureId'] == 'CASTHMA']
asthma = asthma.drop('MeasureId', axis = 1)
asthma = asthma.rename({'Crude_prevalence': "Asthma_prevalence"}, axis=1)
asthma.head()


In [ ]:
asthma.shape


In [ ]:
asthma.describe



In [ ]:
kidney = combined[combined['MeasureId'] == 'KIDNEY']  
kidney = kidney.drop('MeasureId', axis = 1)
kidney = kidney.rename({'Crude_prevalence': "Kidney_prevalence"}, axis=1)
kidney.head()


In [ ]:
kidney.shape


In [ ]:
chd = combined[combined['MeasureId'] == 'CHD']  
chd = chd.drop('MeasureId', axis = 1)
chd = chd.rename({'Crude_prevalence': "CHD_prevalence"}, axis=1)
chd.head()


In [ ]:
stroke = combined[combined['MeasureId'] == 'STROKE']
stroke = stroke.drop('MeasureId', axis = 1)
stroke = stroke.rename({'Crude_prevalence': "Stroke_prevalence"}, axis=1)
stroke.head()


In [ ]:
cancer = combined[combined['MeasureId'] == 'CANCER']
cancer = cancer.drop('MeasureId', axis = 1)
cancer = cancer.rename({'Crude_prevalence': "Cancer_prevalence"}, axis=1)
cancer.head()


In [ ]:
copd = combined[combined['MeasureId'] == 'COPD']
copd = copd.drop('MeasureId', axis = 1)
copd = copd.rename({'Crude_prevalence': "COPD_prevalence"}, axis=1)
copd.head()


 **Environmental risk columns defined**

 PM25 = PM2.5 level in air

 Ozone = Ozone level in air

 DieselPM = Diesel particulate matter level in air

 AirToxicsRisk = Air toxics cancer risk

 RESP = Air toxics respiratory hazard index

 PTRAF = Traffic proximity and volume

 PRE1960PCT = % pre-1960 housing (lead paint indicator)

 PNPL = Proximity to National Priorities List (NPL) [superfund] sites

 PRMP = Proximity to Risk Management Plan (RMP) facilities, facilities that use extremely
        hazardous substances

 PTSDF = Proximity to Treatment Storage and Disposal (TSDF) facilities

 PWDIS = Indicator for major direct dischargers to water


 Making a dataset for each health concern

In [ ]:
copd.to_csv('C:/Users/butle/hp/Capstone_Two/data/interim/copd_data.csv')


In [ ]:
asthma.to_csv('C:/Users/butle/hp/Capstone_Two/data/interim/asthma_data.csv')


In [ ]:
stroke.to_csv('C:/Users/butle/hp/Capstone_Two/data/interim/stroke_data.csv')


In [ ]:
cancer.to_csv('C:/Users/butle/hp/Capstone_Two/data/interim/cancer_data.csv')


In [ ]:
chd.to_csv('C:/Users/butle/hp/Capstone_Two/data/interim/chd_data.csv')


In [ ]:
kidney.to_csv('C:/Users/butle/hp/Capstone_Two/data/interim/kidney_data.csv')


In [ ]:
kidney.to_csv('C:/Users/butle/hp/Capstone_Two/data/interim/kidney_data.csv')


